
# Incorporating OLI Calculations with WaterTAP

#### Contact: Paul Vecchiarelli (paul.vecchiarelli@nrel.gov)

This tutorial will demonstrate basic usage of OLI Cloud calls using our custom API tools.

## Rationale

 - Simulations for realistic water sources are mathematically complex: 
 > $ Interactions \ge Cations * Anions$
 - OLI improves WaterTAP approximations and offloads computational resources

## Required OLI API Inputs


 - State variables (solute concentrations, temperature, pressure), which can be extracted from a state block
 
 - Login credentials
 
 - A chemistry (*.dbs) file
     - establishes state variables, phases, etc. to be considered in flash calls

In [11]:
# used to specify state/convert variables
from pyomo.environ import units as pyunits

# used to build survey lists
from numpy import linspace

# used to execute OLI Cloud functions
from watertap.tools.oli_api.flash import (
    Flash,
    build_survey,
    get_survey_sample_conditions,
    write_output,
)
from watertap.tools.oli_api.credentials import CredentialManager
from watertap.tools.oli_api.client import OLIApi

# 1. Specify State Variables.

- This data is used to construct inputs to OLI Cloud
- Same basic information is required by Water Analysis and Isothermal flashes (temperature, pressure, solutes)

In [3]:
inflows = {
    "Na_+": 3.5e3,
    "Cl_-": 3.5e3,
    "Ca_2+": 250,
    "SO4_2-": 900,
    "SiO2": 30,
}

# 2. Initialize Flash Instance.

 - We will run most of our methods with this class

In [4]:
flash = Flash()

# 3. Get Survey Parameters.

 - In this example, we will generate a temperature sweep survey

In [13]:
# a survey will sweep through one or more variables simultaneously
# surveys are enabled for concentration, pH, temperature, pressure, and other variables

# using the default grid_mesh setting,
# build_survey computes the Cartesian product of the input arrays
survey = build_survey(
    {
        "Na_+": linspace(0, 1e4, 2),
        "Cl_-": linspace(0, 1e4, 5),
    },
    get_oli_names=True,
    mesh_grid=True,
    file_name="pretreatment_survey",
)
print("Survey using grid_mesh")
print(survey)

# setting grid_mesh=False allows custom surveys to be defined, i.e., no Cartesian product is computed.
survey_custom = build_survey(
    {
         "Na_+": [0., 0., 0., 0., 0., 1e4, 1e4, 1e4, 1e4, 1e4],
         "Cl_-": [0., 2.5e3, 5e3, 7.5e3, 1e4, 0., 2.5e3, 5e3, 7.5e3, 1e4],
    },
    get_oli_names=True,
    mesh_grid=False,
    file_name="pretreatment_survey_custom",
)
print("Survey with custom sampling")
print(survey_custom)

OLIAPI - 16:57:20 - INFO - Survey contains 10 items.
OLIAPI - 16:57:20 - INFO - Saving content to pretreatment_survey.json
OLIAPI - 16:57:20 - INFO - Save complete
OLIAPI - 16:57:20 - INFO - Survey contains 10 items.
OLIAPI - 16:57:20 - INFO - Saving content to pretreatment_survey_custom.json
OLIAPI - 16:57:20 - INFO - Save complete


Survey using grid_mesh
{'NAION': [0.0, 0.0, 0.0, 0.0, 0.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0], 'CLION': [0.0, 2500.0, 5000.0, 7500.0, 10000.0, 0.0, 2500.0, 5000.0, 7500.0, 10000.0]}
Survey with custom sampling
{'NAION': [0.0, 0.0, 0.0, 0.0, 0.0, 10000.0, 10000.0, 10000.0, 10000.0, 10000.0], 'CLION': [0.0, 2500.0, 5000.0, 7500.0, 10000.0, 0.0, 2500.0, 5000.0, 7500.0, 10000.0]}


In [14]:
# individual sample points can be accessed to see what will be modified
samples = [0,1,2,3,4,5,6,7,8,9]
survey_points = get_survey_sample_conditions(survey, samples)
survey_points_custom = get_survey_sample_conditions(survey_custom, samples)

for sample in samples:
    print(f"sample: {sample}")
    print(f"grid survey point: {survey_points[sample]}")
    print(f"custom survey point: {survey_points_custom[sample]}")

write_output(survey_points, "test_survey_points")
write_output(survey_points_custom, "test_survey_points_custom")

OLIAPI - 16:57:24 - INFO - Saving content to test_survey_points.json
OLIAPI - 16:57:24 - INFO - Save complete
OLIAPI - 16:57:24 - INFO - Saving content to test_survey_points_custom.json
OLIAPI - 16:57:24 - INFO - Save complete


sample: 0
grid survey point: {'NAION': 0.0, 'CLION': 0.0}
custom survey point: {'NAION': 0.0, 'CLION': 0.0}
sample: 1
grid survey point: {'NAION': 0.0, 'CLION': 2500.0}
custom survey point: {'NAION': 0.0, 'CLION': 2500.0}
sample: 2
grid survey point: {'NAION': 0.0, 'CLION': 5000.0}
custom survey point: {'NAION': 0.0, 'CLION': 5000.0}
sample: 3
grid survey point: {'NAION': 0.0, 'CLION': 7500.0}
custom survey point: {'NAION': 0.0, 'CLION': 7500.0}
sample: 4
grid survey point: {'NAION': 0.0, 'CLION': 10000.0}
custom survey point: {'NAION': 0.0, 'CLION': 10000.0}
sample: 5
grid survey point: {'NAION': 10000.0, 'CLION': 0.0}
custom survey point: {'NAION': 10000.0, 'CLION': 0.0}
sample: 6
grid survey point: {'NAION': 10000.0, 'CLION': 2500.0}
custom survey point: {'NAION': 10000.0, 'CLION': 2500.0}
sample: 7
grid survey point: {'NAION': 10000.0, 'CLION': 5000.0}
custom survey point: {'NAION': 10000.0, 'CLION': 5000.0}
sample: 8
grid survey point: {'NAION': 10000.0, 'CLION': 7500.0}
custom su

WindowsPath('test_survey_points_custom.json')

# 4. Login to OLI Cloud.

- The following code demonstrates an OLI Cloud login:

In [15]:
# there are 3 intended methods of logging in:
# 1: using username, password, root_url, and auth_url
# 2: using access_keys and root_url
# 3: using encryption_key to load credentials from config_file

# methods 1 and 2 will prompt to save credentials and provide an encryption key to use with method 3.

try:
    # method 1: input username, password, root_url, and auth_url
    credential_manager = CredentialManager()
    # when prompted, allow WaterTAP to save credentials to generate encryption key.
    
    # to overwrite saved credentials, create a new credential_manager instance with 
    # the credentials to write and permit credentials to be save to the config_file.
    # e.g., new_credential_manager = CredentialManager(credential_manager.credentials)
    
    
    # method 2: input access_keys and root_url
    
    # credential_manager.generate_oliapi_access_key will create a new access key and add it to
    # credential_manager.credentials["access_keys"].
    
    # credential_manager.delete_oliapi_access_key will delete a specified key.

    """
    credential_manager = CredentialManager(
        access_keys = ["", "", "", "", ""],
        root_url = "",
    )
    """
    
    # method 3: input encryption_key
    """
    credential_manager = CredentialManager(
        encryption_key = "",
    )
    """
    
    # NOTE: a new encryption key will be generated every time the config_file is overwritten 
    
except (OSError, ConnectionError) as e:
    print(e)
    credential_manager = None

 Incomplete credentials for the following keys: ['username', 'password', 'root_url', 'auth_url'].


# 5. Create *.dbs File and 6. Get Output

In [16]:
if credential_manager:
    # OLIApi is used as a context manager
    with OLIApi(credential_manager) as oliapi:
        # create a new DBS file
        # alternative thermo_frameworks and databanks are available.
        
        # use keep_file = True to save a DBS file ID on the Cloud
        # for use in more than one session
        dbs_file_id = oliapi.get_dbs_file_id(
            inflows=inflows,
            thermo_framework="MSE (H3O+ ion)",
            phases=["liquid1", "solid"],
            model_name="test",
            keep_file=true,
        )
        # NOTE: liquid2 (non-aqueous) and vapor phases are available but not yet fully supported.
                
        # if a DBS file has been retained from a previous session,
        # its flash history and chemistry information can be summarized.
        file_summary = oliapi.get_dbs_file_summary(dbs_file_id)
        
        # save chemistry information
        chemistry_info = file_summary["chemistry_info"]
        write_output(chemistry_info["result"], "chemistry_info")
        
        # create water analysis input
        # Water Analysis uses true species for inputs, i.e., IONS.
        json_input = flash.configure_water_analysis(
            inflows,
            reconciliation="ReconcilePh",
            ph=5,
            allow_solids=True,
            file_name="water_analysis_inputs",
        )
        # run Water Analysis flash calculation survey as specified        
        stream_output = flash.run_flash(
            "wateranalysis",
            oliapi,
            dbs_file_id,
            json_input,
            survey,
            file_name="water_analysis_outputs",
        )
        
        # The output of Water Analysis gives apparent species
        # i.e., SALTS, COMPLEXES (MgO, CaO, etc.)

In [ ]:
if credential_manager:
    with OLIApi(credential_manager) as oliapi:

    # Other flash calculations use apparent species for inputs.
    
    # These can be obtained in one step with get_apparent_species_from_true(),
    # and saved to a file and reloaded as needed.
    apparent_species = flash.get_apparent_species_from_true(
        json_input, oliapi, dbs_file_id, file_name="apparent_species"
    )
    isothermal_input = flash.configure_flash_analysis(
        apparent_species, "isothermal", file_name="isothermal_inputs"
    )
    # specify a new survey to compare apparent species
    isothermal_survey = build_survey(
        {"NaCl": linspace(0, 1e6, 10)},
        get_oli_names=True,
        file_name="isothermal_survey",
    )
    flash.run_flash(
        "isothermal",
        oliapi,
        dbs_file_id,
        isothermal_input,
        isothermal_survey,
        file_name="isothermal_outputs",
    )

# 7. Extract Filtered Output

 - OLI's output is robust, so WaterTAP extracts data into a flattened result.

In [20]:
# There are 3 keys for results:
# 'submitted_requests' stores the JSON input for each Flash calculation
# 'metaData' stores metadata for each Flash calculation
# 'result' stores the JSON output for each Flash calculation

import json

if credential_manager:
    with open("isothermal_outputs", "rb") as json_output:
        results = json.load(json_output)["result"]

    print("Available keys:")
    for k in result:
        print(k)

    result = data["prescalingTendencies"]
    print(result)